# Backtesting and Forecasting

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

Load and prepare the data

In [ ]:
def load_and_prepare_data():
    """Load and prepare dengue and SST data."""

    # Load datasets
    DATA_DIR = Path(" Put your data directory path here ")

    dengue_df = pd.read_csv(DATA_DIR / "infodengue_capitals_subsetBR.csv")
    sst_df = pd.read_csv(DATA_DIR / "sst_indices.csv")

    # Parse dates
    dengue_df['data_iniSE'] = pd.to_datetime(dengue_df['data_iniSE'])
    dengue_df['year'] = dengue_df['data_iniSE'].dt.year
    dengue_df['quarter'] = dengue_df['data_iniSE'].dt.quarter
    dengue_df['year_quarter'] = dengue_df['year'].astype(str) + '-Q' + dengue_df['quarter'].astype(str)

    sst_df['date'] = pd.to_datetime(sst_df['YR'].astype(str) + '-' + sst_df['MON'].astype(str) + '-01')
    sst_df['quarter'] = sst_df['date'].dt.quarter
    sst_df['year'] = sst_df['YR']
    sst_df['year_quarter'] = sst_df['year'].astype(str) + '-Q' + sst_df['quarter'].astype(str)

    # Aggregate SST quarterly
    sst_quarterly = sst_df.groupby('year_quarter').agg({
        'NINO1+2': 'mean', 'NINO3': 'mean', 'NINO3.4': 'mean', 'ANOM.3': 'mean',
        'year': 'first', 'quarter': 'first'
    }).reset_index()
    sst_quarterly.columns = ['year_quarter', 'nino12', 'nino3', 'nino34', 'nino34_anom', 'year', 'quarter']

    # Aggregate dengue quarterly - TARGET ONLY (no same-quarter stats = no leakage)
    quarterly = dengue_df.groupby(['year', 'quarter', 'year_quarter']).agg({
        'casos_est': 'sum',
    }).reset_index()

    # Merge
    df = quarterly.merge(sst_quarterly, on='year_quarter', how='left', suffixes=('', '_sst'))
    df['year'] = df['year'].fillna(df['year_sst'])
    df['quarter'] = df['quarter'].fillna(df['quarter_sst'])
    df = df.drop(columns=['year_sst', 'quarter_sst'], errors='ignore')
    df = df.sort_values(['year', 'quarter']).reset_index(drop=True)

    return df

Feature Engineering

In [3]:
def create_features(df):
    """Create features using ONLY past information (no leakage)."""

    # Lag features (1-8 quarters back)
    for lag in [1, 2, 3, 4, 5, 6, 7, 8]:
        df[f'lag_{lag}'] = df['casos_est'].shift(lag)

    # Rolling statistics (from past only - note the shift(1))
    for window in [2, 4, 8]:
        df[f'roll_mean_{window}'] = df['casos_est'].shift(1).rolling(window, min_periods=1).mean()
        df[f'roll_std_{window}'] = df['casos_est'].shift(1).rolling(window, min_periods=1).std()
        df[f'roll_max_{window}'] = df['casos_est'].shift(1).rolling(window, min_periods=1).max()
        df[f'roll_min_{window}'] = df['casos_est'].shift(1).rolling(window, min_periods=1).min()

    # Exponential moving averages
    for span in [2, 4, 8]:
        df[f'ema_{span}'] = df['casos_est'].shift(1).ewm(span=span, adjust=False).mean()

    # Seasonal features (known at forecast time)
    df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'].astype(int) / 4)
    df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'].astype(int) / 4)
    df['is_peak_season'] = ((df['quarter'] == 1) | (df['quarter'] == 2)).astype(int)

    # Year-over-year
    df['yoy_same_quarter'] = df['casos_est'].shift(4)
    df['yoy_growth_rate'] = df['casos_est'].shift(1) / (df['casos_est'].shift(5) + 1)

    # Trend
    df['year_trend'] = df['year'] - df['year'].min()

    # Momentum
    df['momentum_1q'] = df['casos_est'].shift(1) - df['casos_est'].shift(2)
    df['acceleration'] = df['momentum_1q'] - df['momentum_1q'].shift(1)

    # Log transforms
    df['log_lag_1'] = np.log1p(df['casos_est'].shift(1))
    df['log_lag_4'] = np.log1p(df['casos_est'].shift(4))
    df['log_roll_mean_4'] = np.log1p(df['roll_mean_4'])

    # Ratio
    df['ratio_to_roll_mean'] = df['casos_est'].shift(1) / (df['roll_mean_4'] + 1)

    # El Niño lagged
    df['nino34_lag1'] = df['nino34'].shift(1)

    # Get feature columns
    feature_cols = [col for col in df.columns if col not in
                    ['year', 'quarter', 'year_quarter', 'casos_est',
                     'nino12', 'nino3', 'nino34', 'nino34_anom']]

    return df, feature_cols

Train and Evaluate with expanding cross validation and grind search

In [4]:
def train_and_evaluate(df, feature_cols, train_years, test_year):
    """Train model and evaluate on specified test year."""

    # Split data
    train_df = df[df['year'].isin(train_years)].copy()
    test_df = df[df['year'] == test_year].copy()

    # Get valid features
    valid_features = [col for col in feature_cols
                      if train_df[col].notna().sum() > len(train_df) * 0.5]

    # Prepare data
    X_train = train_df[valid_features].fillna(train_df[valid_features].median())
    y_train = train_df['casos_est']
    X_test = test_df[valid_features].fillna(train_df[valid_features].median())
    y_test = test_df['casos_est'].values

    # Test multiple models
    models = {
        'GradientBoosting': GradientBoostingRegressor(
            n_estimators=200, max_depth=4, learning_rate=0.05, random_state=42
        ),
        'RandomForest': RandomForestRegressor(
            n_estimators=300, max_depth=6, min_samples_leaf=2, random_state=42
        ),
        'AdaBoost': AdaBoostRegressor(
            n_estimators=100, learning_rate=0.1, random_state=42
        ),
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = np.maximum(0, model.predict(X_test))
        results[name] = {
            'r2': r2_score(y_test, y_pred),
            'mae': mean_absolute_error(y_test, y_pred),
            'predictions': y_pred,
            'model': model
        }

    return results, y_test, test_df, valid_features, X_train

Bactesting in Normal year and Post-outbreak year

In [5]:
import pandas as pd

def main():
    """Main execution."""

    print("=" * 80)
    print("DENGUE FORECASTING - HONEST MODEL (NO DATA LEAKAGE)")
    print("=" * 80)

    # Load and prepare data
    df = load_and_prepare_data()
    df, feature_cols = create_features(df)

    print(feature_cols)
    print(f"\n Data: {len(df)} quarters, {len(feature_cols)} features")

    # TEST CASE 1: Normal year (2023) - Best case scenario
    print("\n" + "=" * 80)
    print("TEST CASE 1: Predicting 2023 (Normal Year)")
    print("Training: 2010-2022")
    print("=" * 80)

    train_years_1 = list(range(2010, 2023))
    results_2023, y_test_2023, test_df_2023, valid_features, X_train = train_and_evaluate(
        df, feature_cols, train_years_1, 2023
    )

    print(f"\n{'Model':<20} {'R²':>10} {'MAE':>15}")
    print("-" * 48)
    for name, res in results_2023.items():
        print(f"{name:<20} {res['r2']:>10.4f} {res['mae']:>15,.0f}")

    # Best model for 2023
    best_2023 = min(results_2023.items(), key=lambda x: (x[1]['mae'], -x[1]['r2']))
    print(f"\n Best (by MAE): {best_2023[0]} with MAE = {best_2023[1]['mae']:.0f} (R² = {best_2023[1]['r2']:.4f})")

    print("\n 2023 Predictions:")
    print(f"{'Quarter':<12} {'Actual':>15} {'Predicted':>15} {'Error':>15}")
    print("-" * 60)
    for i, (_, row) in enumerate(test_df_2023.iterrows()):
        pred = best_2023[1]['predictions'][i]
        print(
            f"{row['year_quarter']:<12} {y_test_2023[i]:>15,.0f} "
            f"{pred:>15,.0f} "
            f"{(y_test_2023[i] - pred):>15,.0f}"
        )

    # TEST CASE 2: Post-outbreak year (2025)
    print("\n" + "=" * 80)
    print("TEST CASE 2: Predicting 2025 (Post-Outbreak Year)")
    print("Training: 2010-2023 (excluding 2024)")
    print("=" * 80)

    train_years_2 = list(range(2010, 2024))
    results_2025, y_test_2025, test_df_2025, _, _ = train_and_evaluate(
        df, feature_cols, train_years_2, 2025
    )

    print(f"\n{'Model':<20} {'R²':>10} {'MAE':>15}")
    print("-" * 48)
    for name, res in results_2025.items():
        print(f"{name:<20} {res['r2']:>10.4f} {res['mae']:>15,.0f}")

    # Best model for 2025
    best_2025 = min(results_2025.items(), key=lambda x: (x[1]['mae'], -x[1]['r2']))
    print(f"\n Best (by MAE): {best_2025[0]} with MAE = {best_2025[1]['mae']:.0f} (R² = {best_2025[1]['r2']:.4f})")

    print("\n 2025 Predictions:")
    print(f"{'Quarter':<12} {'Actual':>15} {'Predicted':>15} {'Error':>15}")
    print("-" * 60)
    for i, (_, row) in enumerate(test_df_2025.iterrows()):
        pred = best_2025[1]['predictions'][i]
        print(
            f"{row['year_quarter']:<12} {y_test_2025[i]:>15,.0f} "
            f"{pred:>15,.0f} "
            f"{(y_test_2025[i] - pred):>15,.0f}"
        )

    # FEATURE IMPORTANCE
    print("\n" + "=" * 80)
    print(" TOP 10 FEATURE IMPORTANCE (RandomForest for 2023)")
    print("=" * 80)

    rf_model = results_2023['RandomForest']['model']
    importance = pd.DataFrame({
        'Feature': valid_features,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    for _, row in importance.head(10).iterrows():
        bar = '█' * int(row['Importance'] * 50)
        print(f"   {row['Feature']:<25} {row['Importance']:.4f} {bar}")


    # SUMMARY
    print("\n" + "=" * 80)
    print(" SUMMARY")
    print("=" * 80)

    print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│                    HONEST MODEL PERFORMANCE (NO LEAKAGE)                    │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  NORMAL YEAR (2023):                                                        │
│  ├── Best Model (lowest MAE): {best_2023[0]:<20}                            │
│  ├── MAE: {best_2023[1]['mae']:,.0f} cases                                  │
│  └── R² Score: {best_2023[1]['r2']:.4f}                                     │
│                                                                             │
│  POST-OUTBREAK YEAR (2025):                                                 │
│  ├── Best Model (lowest MAE): {best_2025[0]:<20}                            │
│  ├── MAE: {best_2025[1]['mae']:,.0f} cases                                  │
│  └── R² Score: {best_2025[1]['r2']:.4f}                                     │
│                                                                             │
│  WHY 2025 PREDICTION IS POOR:                                               │
│  • 2024 had unprecedented outbreak (877K cases in Q1)                       │
│  • 2025 baseline is 3-5x higher than historical average                     │
│  • Model trained on 2010-2023 cannot extrapolate to "new normal"            │
│                                                                             │
│  FEATURES USED (all lagged - no leakage):                                   │
│  • Lag features: casos_est from quarters 1-8 back                           │
│  • Rolling stats: mean, std, max from past quarters                         │
│  • Seasonal: quarter sin/cos, peak season indicator                         │
│  • Year-over-year: same quarter previous year                               │
│  • Momentum: rate of change between quarters                                │
│  • El Niño: NINO3.4 index (lagged)                                          │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
""")

    print("=" * 80)
    print("ANALYSIS COMPLETE")
    print("=" * 80)

    return results_2023, results_2025, test_df_2025


results_2023, results_2025, test_df_2025 = main()


DENGUE FORECASTING - HONEST MODEL (NO DATA LEAKAGE)
['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'roll_mean_2', 'roll_std_2', 'roll_max_2', 'roll_min_2', 'roll_mean_4', 'roll_std_4', 'roll_max_4', 'roll_min_4', 'roll_mean_8', 'roll_std_8', 'roll_max_8', 'roll_min_8', 'ema_2', 'ema_4', 'ema_8', 'quarter_sin', 'quarter_cos', 'is_peak_season', 'yoy_same_quarter', 'yoy_growth_rate', 'year_trend', 'momentum_1q', 'acceleration', 'log_lag_1', 'log_lag_4', 'log_roll_mean_4', 'ratio_to_roll_mean', 'nino34_lag1']

 Data: 64 quarters, 36 features

TEST CASE 1: Predicting 2023 (Normal Year)
Training: 2010-2022

Model                        R²             MAE
------------------------------------------------
GradientBoosting        -1.1611          34,712
RandomForest             0.6112          14,317
AdaBoost                -0.4104          27,040

 Best (by MAE): RandomForest with MAE = 14317 (R² = 0.6112)

 2023 Predictions:
Quarter               Actual       Predicte

Forcasting of 2026

In [6]:
from sklearn.base import clone
import pandas as pd
import numpy as np

def refit_best_model_and_forecast_2026(df, feature_cols, results_2023, results_2025,
                                      use_best_from="2023",
                                      exclude_2024=True):
    """
    Refit best model on all data up to 2025 (optionally excluding 2024),
    then recursively forecast 2026Q1..2026Q4.
    """

    # Pick best model
    if use_best_from == "2023":
        best_name = max(results_2023.items(), key=lambda x: x[1]['r2'])[0]
        base_model = results_2023[best_name]['model']
    else:
        best_name = max(results_2025.items(), key=lambda x: x[1]['r2'])[0]
        base_model = results_2025[best_name]['model']

    print(f"Using best model from {use_best_from}: {best_name}")


    # Build training set up to 2025
    train_df = df[df["year"] <= 2025].copy()
    if exclude_2024:
        train_df = train_df[train_df["year"] != 2024].copy()

    # Create features on the training df
    train_df_feat, _ = create_features(train_df)

    # Keep only rows where target exists
    train_df_feat = train_df_feat.dropna(subset=["casos_est"]).copy()

    # Keep only valid feature columns that exist
    valid_features = [c for c in feature_cols if c in train_df_feat.columns]

    # Drop rows with any NaN
    X_train = train_df_feat[valid_features].copy()
    y_train = train_df_feat["casos_est"].values

    all_nan_cols = [c for c in valid_features if X_train[c].isna().all()]
    valid_features = [c for c in valid_features if c not in all_nan_cols]
    X_train = train_df_feat[valid_features].copy()

    # Drop rows with NaNs
    mask = ~X_train.isna().any(axis=1)
    X_train = X_train.loc[mask]
    y_train = y_train[mask.values]

    # Refit
    model = clone(base_model)
    model.fit(X_train, y_train)

    # Create future quarters for 2026Q1..Q4
    future_rows = []
    for q in [1, 2, 3, 4]:
        future_rows.append({
            "year": 2026,
            "quarter": q,
            "year_quarter": f"2026-Q{q}",
            "casos_est": np.nan,
            # ENSO columns (scenario fill below)
            "nino12": np.nan,
            "nino3": np.nan,
            "nino34": np.nan,
            "nino34_anom": np.nan,
        })

    future_df = pd.DataFrame(future_rows)

    # Scenario assumption for ENSO in 2026:
    # Use last observed values (persistence) from 2025Q4
    last_obs = df[df["year"] <= 2025].sort_values(["year", "quarter"]).iloc[-1]
    for col in ["nino12", "nino3", "nino34", "nino34_anom"]:
        if col in df.columns and pd.notna(last_obs.get(col, np.nan)):
            future_df[col] = last_obs[col]

    # Combine history + future placeholders
    extended = pd.concat([df.copy(), future_df], ignore_index=True)
    extended = extended.sort_values(["year", "quarter"]).reset_index(drop=True)


    # Recursive forecast for 2026Q1..Q4
    preds = []
    for step in range(4):
        # Recompute features each step so lags/rolling include previous predictions
        ext_feat, _ = create_features(extended)

        # Find current quarter row (2026-Q{step+1})
        yq = f"2026-Q{step+1}"
        row_idx = ext_feat.index[ext_feat["year_quarter"] == yq][0]

        X_row = ext_feat.loc[[row_idx], valid_features].copy()

        medians = X_train.median(numeric_only=True)
        X_row = X_row.fillna(medians)

        y_pred = float(model.predict(X_row)[0])
        preds.append((yq, y_pred))

        extended.loc[extended["year_quarter"] == yq, "casos_est"] = y_pred

    forecast_2026 = pd.DataFrame(preds, columns=["year_quarter", "predicted_casos_est"])
    return forecast_2026, model, valid_features


# RUN FORECAST FOR 2026
df = load_and_prepare_data()
df, feature_cols = create_features(df)

forecast_2026, fitted_model, used_features = refit_best_model_and_forecast_2026(
    df=df,
    feature_cols=feature_cols,
    results_2023=results_2023,
    results_2025=results_2025,
    use_best_from="2023",
    exclude_2024=True
)

print("\n 2026 Quarterly Forecast (4 steps ahead):")
print(forecast_2026)


Using best model from 2023: RandomForest

 2026 Quarterly Forecast (4 steps ahead):
  year_quarter  predicted_casos_est
0      2026-Q1         81411.121861
1      2026-Q2         86399.481258
2      2026-Q3         29230.254434
3      2026-Q4         29297.428339
